# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

## If you'd prefer to work in IDEs

If you're more comfortable in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  
If you'd prefer to work in VSCode, [here](https://chatgpt.com/share/676f2e19-c228-8012-9911-6ca42f8ed766) are instructions from an AI friend on how to configure it for the course.

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [15]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [16]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [19]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [20]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome! I’m so glad you’re here. How can I assist you today?


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [10]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [12]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [16]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the might GPT (yet!)

In [13]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [14]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, you know, just a casual 4. But if you’re looking for something more exciting, you could always say it’s “a pair of twos having a meeting.”


## And now let's build useful messages for GPT-4o-mini, using a function

In [17]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [18]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

## Time to bring it together - the API for OpenAI is very simple!

In [19]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [20]:
summarize("https://edwarddonner.com")

'# Summary of Edward Donner\'s Website\n\nEdward Donner\'s website showcases his interests and professional background in artificial intelligence (AI) and large language models (LLMs). He is the co-founder and CTO of Nebula.io, which focuses on utilizing AI to help individuals discover their potential in the talent management field. He has a history of founding AI startups, including untapt, which was acquired in 2021.\n\n## Recent Announcements and Posts\n1. **December 21, 2024** - Welcome message to SuperDataScientists.\n2. **November 13, 2024** - Resources for mastering AI and LLM engineering.\n3. **October 16, 2024** - Guidance and resources for transitioning from software engineer to AI data scientist.\n4. **August 6, 2024** - Introduction to "Outsmart," an arena where LLMs compete in diplomacy and strategy.'

In [23]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [25]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal and professional platform where Ed shares his passion for coding and experimentation with LLMs (Large Language Models). He is the co-founder and CTO of Nebula.io, an AI company focused on aiding talent discovery and management, and previously founded the AI startup untapt.

### Key Features:

- **Outsmart Arena**: A competitive environment designed for LLMs to engage in strategic diplomacy and cunning tactics.
- **About Ed**: Ed expresses his interests in coding, DJing, and electronic music, as well as his engagement with the tech community through platforms like Hacker News.

### Recent Announcements:
- **December 21, 2024**: Welcoming "SuperDataScientists."
- **November 13, 2024**: Sharing resources for mastering AI and LLM engineering.
- **October 16, 2024**: Resources for transitioning from a software engineer to an AI data scientist.
- **August 6, 2024**: Launch of the Outsmart LLM Arena.

Ed encourages visitors to connect for further discussions and insights.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [26]:
display_summary("https://cnn.com")

# Summary of CNN Website

CNN's website serves as a comprehensive platform for breaking news and updates on various topics including politics, business, health, entertainment, and global events. It features live updates, articles, videos, and special editorial content covering significant current events and investigations.

## Key News Highlights:
- **Political Updates:** Congress certified Trump's election win, marking a comeback in U.S. politics. Trump is also seeking a delay in his sentencing related to a hush money conviction.
- **Global Events:** Canada's Prime Minister Justin Trudeau announced his resignation, expressing regret about his leadership role.
- **Winter Weather:** A major winter storm has affected DC, causing power outages and travel disruptions, with forecasts for additional snow.
- **Legal Issues:** A lawsuit has been filed against an ex-Fox Sports host, alleging harassment. In Brazil, a relative was arrested following a poisoning case that killed three women.
- **Health Concerns:** The FDA's newly set lead limits in baby food have disappointed advocates, and there are rising cases of norovirus nationwide.

Overall, CNN remains a crucial source for timely reporting on national and international developments.

In [28]:
display_summary("https://anthropic.com")

# Summary of Anthropic

Anthropic is an AI research and safety company based in San Francisco, focusing on creating reliable and beneficial AI systems. Their flagship product, Claude, has multiple versions, including the newly released **Claude 3.5 Sonnet** and **Claude 3.5 Haiku**, which are designed to enhance user interaction through conversational AI.

## Key Features:
- **Claude 3.5 Sonnet**: The most advanced AI model offered by Anthropic.
- **API Access**: Developers can build custom applications and experiences using Claude.

## Recent Announcements:
- **New Model Releases** (Oct 22, 2024): Introduction of Claude 3.5 Sonnet and Haiku, along with capabilities for computer use.
- **Product Updates**: Continuous enhancements and updates to their model lineup.
- **Research Initiatives**: Focus on AI safety, including discussions on Constitutional AI and alignment with safety standards.

Anthropic emphasizes the importance of safety in AI development and actively seeks to improve the ethical implications of AI technology through interdisciplinary research and collaboration.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [37]:
# Step 1: Create your prompts

system_prompt = "You are an arrogant assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Be sure to respond in MARKDOWN."


ts = Website("https://twosetviolin.com/about")
# print(ts.title)
# print(ts.text)

user_prompt = f"""
    You are looking at a website with the title {ts.title} \n \
    The contents of this website are as follows {ts.text} \
    please provide a short summary of this website in markdown. \
    If it includes news or announcements, then summarize these too. \ 
    Get rid of logs, buttons, footer, or anything like that.
"""

# See how this function creates exactly the format above

def my_messages_for(system_prompt, website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# Step 2: Make the messages list

messages = my_messages_for(system_prompt, ts) # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )

# Step 4: print the result
display(Markdown(response.choices[0].message.content))

# TwoSet Violin Summary

TwoSet Violin is an Australian classical music duo, Brett & Eddy, celebrated for their innovative approach to making classical music accessible and engaging for today's audience. With over 9.5 million social media followers and one billion views, they blend humor and relatability with their performances, which has inspired musicians worldwide. 

### Key Highlights:
- **Background:** Former members of the Sydney and Queensland Symphony Orchestras, they transitioned to live performances in 2016, launching a unique blend of classical music and comedy.
- **Performances:** Their concerts create an interactive experience that appeals to a wide range of audiences, including those new to classical music.
- **World Tour:** In 2017, they embarked on the first crowdfunded classical world tour, performing in various major cities globally and changing how classical music is perceived.
- **Mission:** They aim to broaden the reach of classical music and inspire future generations through their entertaining and lighthearted approach.

For updates and future projects, visitors can sign up on their website.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException

In [21]:
chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(options=chrome_options)

# visit the target website
driver.get("https://quotes.toscrape.com")
quotes = []

def scraper():
    # get the product containers
    qoute_contrainer = driver.find_elements(By.CLASS_NAME, "quote")

    # extract the product names and prices
    for qoute in qoute_contrainer:
        content = qoute.find_element(By.CLASS_NAME, "text")
        print(content.text)
        author = next_page_link.find_element(By.TAG_NAME, "author")
        print(author.text)
        quotes.append({"text": content.text, "autor": author.text})
        



for i in range(0, 5):

    # execute the scraper function
    scraper()
    

    try:
        # find and click the next page link
        next_page_link = driver.find_element(By.CLASS_NAME, "next")
        a_element = next_page_link.find_element(By.TAG_NAME, "a")
        if a_element:
            a_element.click()
    except NoSuchElementException:
        print("No more pages available")
        break

driver.quit()


“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
Albert Einstein
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
J.K. Rowling
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
Albert Einstein
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
Jane Austen
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
Marilyn Monroe
“Try not to become a man of success. Rather become a man of value.”
Albert Einstein
“It is better to be hated for what you are than to be loved for what you are not.”
André Gide
“I have not failed. I've just found 10,000 ways that won't work.”
Thomas A. Edison
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
Eleanor Roosevelt
“A day witho

In [24]:
# Step 1: Create your prompts

system_prompt = "You are an arrogant assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Be sure to respond in MARKDOWN."

topic = "love"
user_prompt = f"""
    You are looking at a website with the following quotes {quotes} \n \
    please provide the top 3 quotes about {topic} of this website in markdown with their associated authors. \
"""

# See how this function creates exactly the format above

def my_messages_for(system_prompt, website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

# Step 2: Make the messages list

messages = my_messages_for(system_prompt, user_prompt) # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages
    )

# Step 4: print the result
display(Markdown(response.choices[0].message.content))

Here are the top 3 quotes about love from the website:

1. **“I love you without knowing how, or when, or from where. I love you simply, without problems or pride: I love you in this way because I do not know any other way of loving but this, in which there is no I or you, so intimate that your hand upon my chest is my hand, so intimate that when I fall asleep your eyes close.”**  
   — *Pablo Neruda*

2. **“The truth is, everyone is going to hurt you. You just got to find the ones worth suffering for.”**  
   — *Bob Marley*

3. **“Love does not begin and end the way we seem to think it does. Love is a battle, love is a war; love is a growing up.”**  
   — *James Baldwin*

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c